In [41]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 89.1/89.1 MB 32.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./data/adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape



import pandas as pd

# =====EDA=====
# (26048, 15) (6513, 15) (26048, 2) (6513, 2)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# Classifier, >50K, <=50K
# print(y_train.head())
# workclass(1456), occupation(1463), native.country(461)
# print(X_train.isnull().sum())
# workclass(380), occupation(380), native.country(122)
# print(X_test.isnull().sum())
# 0
# print(y_train.isnull().sum())
# 0
# print(y_test.isnull().sum())
# print(X_train.head(3))

df = pd.concat([X_train, X_test])
df = df.drop(columns=['education'])
df = df.drop(columns=['marital.status'])
df = df.drop(columns=['capital.gain'])
df = df.drop(columns=['capital.loss'])
df = df.drop(columns=['native.country'])

df = df.set_index('id')
y_train = y_train.set_index('id')
y_test = y_test.set_index('id')
# print(df.head(3))
# print('='*30)

# =====결측치=====
# print(df[['workclass', 'occupation', 'native.country']])
# print(df.describe(include='object'))
# print(df.isnull().sum())
df['workclass'] = df['workclass'].fillna('Private')
df['occupation'] = df['occupation'].fillna('Prof-specialty')
# print(df.isnull().sum())

# =====이상치 확인=====
# print(df.describe())

# =====스케일링=====
# print(df.describe())
from sklearn.preprocessing import minmax_scale
cols = df.describe().columns
for col in cols:
    df[col] = minmax_scale(df[col])
# print(df.describe())

# =====원핫인코딩=====
df = pd.get_dummies(df)
# print(df.describe())

X_train = df[:X_train.shape[0]]
X_test = df[X_train.shape[0]:]

# =====라벨 인코더=====
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train['income'] = le.transform(y_train)
y_test['income'] = le.transform(y_test)
# print(y_train.head())
# print(dir(le))
# print(le.classes_)

# =====train_test_split=====
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)

# =====모델 생성=====
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2022)

# =====모델 fit=====
model.fit(X_train, y_train)

# =====모델 성능 평가=====
from sklearn.metrics import roc_auc_score
print(model.score(X_val, y_val))
print(roc_auc_score(model.predict(X_val), y_val))
print('='*30)

# XGB
# import xgboost
# print(dir(xgboost))
# from xgboost import XGBClassifier
# model = XGBClassifier()
# model.fit(X_train, y_train)
# print(model.score(X_val, y_val))
# print(roc_auc_score(model.predict(X_val), y_val))
# 거의 차이 없음

# =====모델 predict=====
pred = model.predict(X_test)

# =====submit=====
submit = pd.DataFrame()
submit['id'] = y_test.index

res = []
for i in pred:
    res.append(le.classes_[i])
submit['income'] = res

submit.to_csv('adult_pred.csv', index=False)

submit = pd.read_csv('adult_pred.csv')
print(submit.head(3))
print('='*30)
print(model.score(X_test, y_test))
print(roc_auc_score(model.predict(X_test), y_test))

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hjk70\AppData\Local\Temp\ipykernel_3280\1170176737.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


0.8228406909788868
0.762091102986854
      id income
0  20901   >50K
1  14170  <=50K
2   1776  <=50K
0.8257331490864425
0.7627917879483161
